In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../scenarios/challenge.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
#change some colors
RealWorld.getFrame("obj0").setColor([0,1,0])
RealWorld.getFrame("obj1").setColor([0,1,0])
RealWorld.getFrame("obj2").setColor([1,1,0])
RealWorld.getFrame("obj3").setColor([1,0,1])
RealWorld.getFrame("obj4").setColor([0,1,1])

#you can also change the shape & size
RealWorld.getFrame("obj0").setColor([1.,0,0])
RealWorld.getFrame("obj0").setShape(ry.ST.sphere, [.03])
#RealWorld.getFrame("obj0").setShape(ry.ST.ssBox, [.05, .05, .2, .01])
RealWorld.getFrame("obj0").setPosition([0., .2, 2.])

#remove some objects
for o in range(5,30):
    name = "obj%i" % o
    #print("deleting", name)
    RealWorld.delFrame(name)

V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

In [4]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [5]:
# we're adding an "imp" to the simulation, which is a little process that can inject perturbations
# S.addImp(ry.ImpType.objectImpulses, ['obj0'], [])

In [6]:
# create your model world
C = ry.Config()
C.addFile('../../scenarios/pandasTable.g')
#V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

In [7]:
def preception(rgb):
    # Segment Red sphere
    # Load an color image as HSV
    # converting from BGR to HSV color space
    hsv_img = cv.cvtColor(rgb, cv.COLOR_BGR2HSV)
        
    # Filter the color of the image to find all pixels that are redish.

    # Threshold the HSV image, keep only the red pixels
    red_lowerth = np.array([0,100,90])
    red_upperth = np.array([10,255,255])
    red_mask1 = cv.inRange(hsv_img, red_lowerth, red_upperth)

    # Range for upper range
    red_lowerth = np.array([160,100,90])
    red_upperth = np.array([180,255,255])
    red_mask2 = cv.inRange(hsv_img,red_lowerth,red_upperth)
    
    # Generating the final mask to detect red color and find all pixels that are 
    red_mask = cv.addWeighted(red_mask1, 1, red_mask2, 1, 0)
    #red_mask = cv.GaussianBlur(red_mask, (9, 9), 0)
    cv.imshow("Masked_Image",red_mask)
    
    # Get only the circle 
        
    # Set my output imgage pixel value to zero everywhere except my mask
    red_obj_img = rgb.copy()
    red_obj_img[np.where(red_mask==0)] = 0
    #cv.imshow("Masked_Image",red_obj_img)
    
    """
    # Detect circles using Hough Transform - Not Working
    # void cv::HoughCircles(InputArray image, int method, double dp, double minDist, 
    #                       double param1 = 100, double param2 = 100, int minRadius = 0, int maxRadius = 0)
    
    height, width = red_mask.shape
    # Debug
    # print("Height - ", height, "Width - ", width)
    # circles = cv.HoughCircles(red_mask, cv.HOUGH_GRADIENT, 1, (width*height)/8, 80, 50, 1)
        
    circles = cv.HoughCircles(red_mask,cv.HOUGH_GRADIENT,1,(width*height)/8,param1=50,param2=50,minRadius=5,maxRadius=0)
    print(circles)
    
    """
    
    """
    # Detect circles using contour
    # Find contours for the objects detected in mask
    # contours, hierarchy=cv.findContours(image, mode, method[, contours[, hierarchy[, offset]]])
    contours, _ = cv.findContours(red_mask, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    red_mask_ret = red_mask.copy()
    for contour in contours:
        # If contour area too small ignore it
        # if abs(cv.contourArea(contour)) < 10000:
            # continue
        
        # Approximate to a polygon of less vertices
        peri = cv.arcLength(contour, True)
        # void cv::approxPolyDP (InputArray curve, OutputArray approxCurve, double epsilon, bool closed)
        approx = cv.approxPolyDP(contour, 0.04 * peri, True)
        
        print("approx", len(approx))
        
        # Choose first circle, can be adapted later and  
        if len(approx) < 7 and not cv.isContourConvex(approx):
            for i in range(red_mask_ret.shape[0]):
                for j in range(red_mask_ret.shape[1]):
                    if ( cv.pointPolygonTest(contour, (i,j), False) < 0):
                        red_mask_ret[i,j] = 0          
            break
    
    cv.imshow("Masked_Image",red_mask_ret)
    """
    
    return red_mask, red_obj_img
    

In [8]:
points = []
cll_point = []

tau = .01

# Initial seen depth and RGB for default point cloud
[rgb_init, depth_init] = S.getImageAndDepth()
init_points = S.depthData2pointCloud(depth_init, fxfypxpy)

for t in range(400):
    time.sleep(0.01)
            
    #grab sensor readings from the simulation
    q = S.get_q()
    if t%10 == 0:
        
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        
        # Changes for OpenCV dispaly
        rgb = cv.cvtColor(rgb, cv.COLOR_BGR2RGB)
        
        #points = S.depthData2pointCloud(depth, fxfypxpy)
        #cameraFrame.setPointCloud(points, rgb)
        #V.recopyMeshes(C)
        #V.setConfiguration(C)
        
        #np.argwhere(img == 255)
            
        if len(rgb)>0: cv.imshow('OPENCV - rgb', rgb)
        if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

        # Start the pipeline
        
        # Stage 1: Perception Pipeline
        red_mask, red_obj = preception(rgb)
        red_obj = cv.cvtColor(red_obj, cv.COLOR_RGB2BGR)
        
        # collect all image points that belong to the object into a n-times-3 matrix with rows
        img_point_rgb = rgb_init.copy()
        img_point_depth = depth_init.copy()
        
        # Debug
        #print(np.array(img_point_rgb).shape)
        #print(np.array(img_point_depth).shape)
        
        for x in range(red_mask.shape[0]):
            for y in range(red_mask.shape[1]):
                if(red_mask[x,y] != 0):
                    img_point_rgb[x,y] = red_obj[x,y]
                    img_point_depth[x,y] = depth[x,y]
                    #cll_rgb.append(img_points[x,y])
                    #cll_depth.append(img_points[x,y])
        
        # Transform that point cloud into meter coordinates
        img_points = S.depthData2pointCloud(img_point_depth, fxfypxpy)
        
        for x in range(red_mask.shape[0]):
            for y in range(red_mask.shape[1]):
                if(red_mask[x,y] != 0):
                    cll_point.append(img_points[x,y])
                         
        print(np.array(cll_point).shape)
                    
        # Display this point could by attaching it as shape to the cameraFrame
        cameraFrame.setPointCloud(img_points, img_point_rgb)
        V.recopyMeshes(C)
        V.setConfiguration(C)
        
        # Compute the mean of the point cloud
        point_mean = np.mean(cll_point, axis=0)
        
        # Debug
        #print("Mean is ", point_mean)
        
        """
        # Path
        IK = C.komo_IK(False)
        
        # addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
        # To prevent collisions
        IK.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e1])

        # Position difference between gripper centre is 0
        IK.addObjective([1], ry.FS.positionDiff, ['R_gripperCenter', point_mean], ry.OT.sos, [1e1])

        # Calling the optimizer (True means random initialization/restart)
        IK.optimize(True)
        IK.getReport()

        qT = IK.getConfiguration(0)
        q0 = C.getFrameState()

        MP = (1 - math.cos(math.pi * t / 250)) / 2
        q = q0 + MP * (qT - q0)
        C.setFrameState(q)
        V.setConfiguration(C)
        
        # Assuming at 250 u reach the object. Should be done only when reached the goal position. 
        if t==250:
        S.closeGripper("gripper")
        """
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
        
    S.step([], tau, ry.ControlMode.none)

(0,)


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(0,)
(0,)
(0,)
(533, 3)
(763, 3)
(980, 3)
(1185, 3)
(1394, 3)
(1601, 3)
(1808, 3)
(2014, 3)
(2223, 3)
(2432, 3)
(2637, 3)
(2842, 3)
(3047, 3)
(3256, 3)
(3464, 3)
(3670, 3)
(3881, 3)
(4088, 3)
(4293, 3)
(4506, 3)
(4714, 3)
(4923, 3)
(5136, 3)
(5345, 3)
(5557, 3)
(5771, 3)
(5982, 3)
(6196, 3)
(6410, 3)
(6622, 3)
(6836, 3)
(7052, 3)
(7266, 3)
(7483, 3)
(7699, 3)
(7913, 3)


In [ ]:
cv.destroyAllWindows()

In [ ]:
# Set contacts for coliision handling

#Contact for table
Table = C.frame("table")
Table.setContact(1)

# Contact for Right gripper and fingers
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
R_gripperCenter = C.frame("R_gripperCenter")
R_gripperCenter.setContact(1)
R_finger1 = C.frame("R_finger1")
R_finger1.setContact(1)
R_finger2 = C.frame("R_finger2")
R_finger2.setContact(1)


# Contact for Left gripper
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)
L_finger1 = C.frame("L_finger1")
L_finger1.setContact(1)
L_finger2 = C.frame("L_finger2")
L_finger2.setContact(1)


# Contact for object
obj0.setContact(1)

In [ ]:
V = komo.view()

In [ ]:
V.playVideo()

In [ ]:
C.setFrameState(komo.getConfiguration(19))
C.getJointState()
V.setConfiguration(C)
coll = C.feature(ry.FS.accumulatedCollisions, [])
C.computeCollisions() 
coll.eval(C)